# RAG with Elastic and Llama3 using Llamaindex

This interactive notebook uses `Llamaindex` to process fictional workplace documents and uses `Llama3` running locally using `Ollama` to transform these documents into embeddings and store them into `Elasticsearch`. We then ask a question, retrieve the relevant documents from `Elasticsearch` and use `Llama3` to provide a response. 

**_Note_** : _Llama3 is expected to be running using `Ollama` on the same machine where you will be running this notebook._

## Requirements

For this example, you will need:

- An Elastic deployment
  - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))
  - For LLM we will be using [Ollama](https://ollama.com/) and [Llama3](https://ollama.com/library/llama3) configured locally.  

### Use Elastic Cloud

If you don't have an Elastic Cloud deployment, follow these steps to create one.

1. Go to [Elastic cloud Registration](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) and sign up for a free trial
2. Select **Create Deployment** and follow the instructions

## Install required dependencies for LlamaIndex and Elasticsearch

First we install the packages we need for this example.

In [ ]:
!pip install llama-index llama-index-cli llama-index-core llama-index-embeddings-elasticsearch llama-index-embeddings-ollama llama-index-legacy llama-index-llms-ollama llama-index-readers-elasticsearch llama-index-readers-file llama-index-vector-stores-elasticsearch llamaindex-py-client

## Import packages
Next we import the required packages as required. The imports are placed in the cells as required.

## Prompt user to provide Cloud ID and API Key
We now prompt the user to provide us Cloud ID and API Key using `getpass`. We get these details from the deployment. 

In [ ]:
from getpass import getpass


# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

## Prepare documents for chunking and ingestion
We now prepare the data to be in the [Document](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/) type for processing using [Llamaindex](https://docs.llamaindex.ai/en/stable/) 

In [6]:
import json
from urllib.request import urlopen
from llama_index.core import Document

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/workplace-documents.json"

response = urlopen(url)
workplace_docs = json.loads(response.read())

# Building Document required by LlamaIndex.
documents = [
    Document(
        text=doc["content"],
        metadata={
            "name": doc["name"],
            "summary": doc["summary"],
            "rolePermissions": doc["rolePermissions"],
        },
    )
    for doc in workplace_docs
]

## Define Elasticsearch and ingest pipeline in LlamaIndex for document processing. Use Llama3 for generating embeddings.
We now define the `Elasticsearchstore` with the required index name, the text field and its associated embeddings. We use `Llama3` to generate the embeddings. We will be running Semantic search on the index to find documents relevant to the query posed by the user. We will use the `SentenceSplitter` provided by `Llamaindex` to chunk the documents. All this is run as part of an `IngestionPipeline` provided by the `Llamaindex` framework.

In [7]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

es_vector_store = ElasticsearchStore(
    index_name="workplace_index",
    vector_field="content_vector",
    text_field="content",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)
# Embedding Model to do local embedding using Ollama.
ollama_embedding = OllamaEmbedding("llama3")
# LlamaIndex Pipeline configured to take care of chunking, embedding
# and storing the embeddings in the vector store.
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=100),
        ollama_embedding,
    ],
    vector_store=es_vector_store,
)

## Execute pipeline 
This will chunk the data, generate embeddings using `Llama3` and ingest into `Elasticsearch` index, with embeddings in a `dense` vector field.

In [ ]:
pipeline.run(show_progress=True, documents=documents)

The embeddings are stored in a dense vector field of dimension `4096`. The dimension size comes from the size of the embeddings generated from `Llama3`.

## Define LLM settings. 
This connects to your local LLM. Please refer to https://ollama.com/library/llama3 for details on steps to run Llama3 locally. 

_If you have sufficient resources (atleast >64 GB Ram and GPU available) then you could try the 70B parameter version of Llama3_ 

In [9]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

Settings.embed_model = ollama_embedding
local_llm = Ollama(model="llama3")

### Setup Semantic search and integrate with Llama3. 
We now configure `Elasticsearch` as the vector store for the `Llamaindex` query engine. The query engine, using `Llama3` is then used to answer your questions with contextually relevant data from `Elasticsearch`.

In [ ]:
from llama_index.core import VectorStoreIndex, QueryBundle

index = VectorStoreIndex.from_vector_store(es_vector_store)
query_engine = index.as_query_engine(local_llm, similarity_top_k=10)

# Customer Query
query = "What are the organizations sales goals?"
bundle = QueryBundle(
    query_str=query, embedding=Settings.embed_model.get_query_embedding(query=query)
)

response = query_engine.query(bundle)

print(response.response)

_You could now try experimenting with other questions._